In [3]:
import pandas as pd
import numpy as np
import itertools
import copy
import random

In [32]:
tokenized = open("Tokenized-2Q1.txt", "r", encoding="utf-8")
Para = tokenized.read()
Para = Para.split("\n")
remove = ['‘', '?', '.', '’', ',']

for sentence in range(len(Para)):
    Para[sentence] = Para[sentence].split(" ")

    for word in range(len(Para[sentence])):
        if "\u200c" in Para[sentence][word]:
            Para[sentence][word] = " ".join(Para[sentence][word].split("\u200c"))

        clean = ""
        for ch in Para[sentence][word]:
            if ch not in remove:
                clean += ch
        Para[sentence][word] = clean.lower()

In [33]:
def NGram(Gram: int = 1, Paragraph: list = None, Smoothing = None):
    L_List = []
    G_List = []

    if Paragraph is None:
        return

    # --- Unigrams ---
    if Gram >= 1:
        Gram_1 = {}
        total = 0
        for sentence in Paragraph:
            for word in sentence:
                Gram_1[word] = Gram_1.get(word, 0) + 1
                total += 1

        Count_1 = copy.deepcopy(Gram_1)
        for k in Gram_1:
            Gram_1[k] = Gram_1[k] / total

        L_List.append(Count_1)
        G_List.append(Gram_1)

    # --- Bigrams ---
    if Gram >= 2:
        Gram_2 = {}
        for sentence in Paragraph:
            for i in range(len(sentence)-1):
                s = tuple(sentence[i:i+2])
                Gram_2[s] = Gram_2.get(s, 0) + 1

        Count_2 = copy.deepcopy(Gram_2)
        for k in Gram_2:
            Gram_2[k] = Gram_2[k] / Count_1.get(k[0], 1)

        L_List.append(Count_2)
        G_List.append(Gram_2)

    # --- Trigrams ---
    if Gram >= 3:
        Gram_3 = {}
        for sentence in Paragraph:
            for i in range(len(sentence)-2):
                s = tuple(sentence[i:i+3])
                Gram_3[s] = Gram_3.get(s, 0) + 1

        Count_3 = copy.deepcopy(Gram_3)
        for k in Gram_3:
            Gram_3[k] = Gram_3[k] / Count_2.get(k[:2], 1)

        L_List.append(Count_3)
        G_List.append(Gram_3)

    # --- Quadgrams ---
    if Gram >= 4:
        Gram_4 = {}
        for sentence in Paragraph:
            for i in range(len(sentence)-3):
                s = tuple(sentence[i:i+4])
                Gram_4[s] = Gram_4.get(s, 0) + 1

        Count_4 = copy.deepcopy(Gram_4)
        for k in Gram_4:
            Gram_4[k] = Gram_4[k] / Count_3.get(k[:3], 1)

        L_List.append(Count_4)
        G_List.append(Gram_4)

    # --- Apply Smoothing if Provided ---
    if Smoothing is not None:
        L1, G1 = L_List[0], G_List[0]
        L2 = L3 = L4 = None
        G2 = G3 = G4 = None
        if Gram >= 2:
            L2, G2 = L_List[1], G_List[1]
        if Gram >= 3:
            L3, G3 = L_List[2], G_List[2]
        if Gram >= 4:
            L4, G4 = L_List[3], G_List[3]

        Smoothing(Gram=Gram, L1=L1, L2=L2, L3=L3, L4=L4,
                  G1=G1, G2=G2, G3=G3, G4=G4, Data=None)

    return L_List, G_List


In [50]:
def Good_Turing_Smoothing_Known(Gram=1, L1=None, L2=None, L3=None, L4=None, G1=None, G2=None, G3=None, G4=None, Data=None):
    # Select the correct dictionary
    L = {1: L1, 2: L2, 3: L3, 4: L4}[Gram]

    if Data not in L:
        return None

    C_MLE = L[Data]
    Nc = sum(1 for val in L.values() if val == C_MLE)
    Nc_1 = sum(1 for val in L.values() if val == C_MLE + 1)

    if Nc == 0:
        C_star = 1  # fallback if no Nc found
    elif Nc_1 == 0:
        C_star = C_MLE  # fallback if no Nc+1 found
    else:
        C_star = (C_MLE + 1) * (Nc_1 / Nc)

    # ensure non-zero smoothing (tiny default)
    if C_star == 0:
        C_star = 0.1

    return (C_MLE, Nc, Nc_1, C_star)


In [35]:
def Good_Turing_Smoothing(Gram=1, L1=None, L2=None, L3=None, L4=None, G1=None, G2=None, G3=None, G4=None, Data=None):
    Dict = {}
    print(Data)
    if Data is None:
        return -1

    L = {1: L1, 2: L2, 3: L3, 4: L4}[Gram]

    words_in_data = []
    for sentence in Data:
        for word in sentence:
            words_in_data.append(word)
    U = set(words_in_data)

    V = len(L1)

    N = sum(L1.values()) if Gram == 1 else sum(L2.values()) if Gram == 2 else sum(L3.values()) if Gram == 3 else sum(L4.values())
    N1 = sum(1 for val in (L1.values() if Gram == 1 else L2.values() if Gram == 2 else L3.values() if Gram == 3 else L4.values()) if val == 1)

    if Gram == 1:
        P = N1 / (N * (V - len(U)))
        for word in U:
            if word not in G1.keys():
                G1[word] = P
    else:
        V_n = len(L1.keys()) ** Gram
        P = (N1 / N) / (V_n - N)

        if Gram == 2:
            for sentence in Data:
                for i in range(len(sentence) - 1):
                    bigram = tuple(sentence[i:i + 2])
                    if bigram not in G2.keys():
                        Dict[bigram] = P
                    else:
                        Dict[bigram] = L[bigram]

        elif Gram == 3:
            for sentence in Data:
                for i in range(len(sentence) - 2):
                    trigram = tuple(sentence[i:i + 3])
                    if trigram not in G3.keys():
                        Dict[trigram] = P
                    else:
                        Dict[trigram] = L[trigram]

        elif Gram == 4:
            for sentence in Data:
                for i in range(len(sentence) - 3):
                    quadrigram = tuple(sentence[i:i + 4])
                    if quadrigram not in G4.keys():
                        Dict[quadrigram] = P
                    else:
                        Dict[quadrigram] = L[quadrigram]

    return Dict

In [36]:
def Kneser(Gram=1, Delta=0.7, L1=None, L2=None, L3=None, L4= None, G1=None, G2=None, G3=None, G4=None, Data=None):
    L = {1: L1, 2: L2, 3: L3, 4: L4}[Gram]

    P1 = max(L[Data] - Delta, 0)
    Before = Data[:-1]

    if P1 != 0:
        Sum = sum(L[x] for x in L.keys() if x[:-1] == Before)
        if Sum > 0:
            P1 /= Sum  # normalize
        else:
            P1 = 0

    Totalities = [L[x] for x in L.keys() if x[:-1] == Before]
    Lambda = len(Totalities) / sum(Totalities) if sum(Totalities) > 0 else 0
    del Totalities

    P2 = len([1 for x in L.keys() if x[-1] == Data[-1]]) / len(L) if len(L) > 0 else 0

    Pkn = P1 + Lambda * P2
    return Pkn


In [48]:
def Katz(Gram=1, L1=None, L2=None, L3=None, L4=None, 
         G1=None, G2=None, G3=None, G4=None, Data=None):

    GT = Good_Turing_Smoothing_Known(
        Gram=Gram, L1=L1, L2=L2, L3=L3, L4=L4,
        G1=G1, G2=G2, G3=G3, G4=G4, Data=Data
    )

    print(GT)

    L = {1: L1, 2: L2, 3: L3, 4: L4}[Gram]
    History = sum([L[x] for x in L.keys() if Data[:-1] == x[:-1]])

    if Data in L:
        dc = GT[0] / GT[-1]  # Good-Turing discount
        P = L[Data]
        Katz = (P / History) * dc
    else:
        # Step 1: Compute leftover probability mass Beta
        Beta = 1
        for x in L.keys():
            if x[:-1] == Data[:-1]:
                GT = Good_Turing_Smoothing_Known(
                    Gram=Gram, L1=L1, L2=L2, L3=L3, L4=L4,
                    G1=G1, G2=G2, G3=G3, G4=G4, Data=x
                )
                dc = GT[0] / GT[-1]
                Beta -= dc * (L[x] / History)

        # Step 2: Generate unseen n-grams by appending each unigram to history
        H = list(Data[:-1])
        List_of_unseen = [tuple(H + [word]) for word in L1.keys()]

        # Step 3: Compute denominator = sum of lower-order probabilities for unseen n-grams
        Lower = 0
        for item in List_of_unseen:
            w = tuple(item)  # ensure tuple
            if w in L.keys():
                Lower += L[w]
            else:
                i = Gram
                while i > 0:
                    i -= 1
                    w_lower = tuple(w[:i])
                    Backoff = {1: L1, 2: L2, 3: L3, 4: L4}[i]
                    if w_lower in Backoff.keys():
                        Lower += Backoff[w_lower]
                        break

        # Step 4: Compute alpha
        Alpha = Beta / max(Lower, 1e-12)

        # Step 5: Get lower-order probability for target word
        w = Data
        i = Gram - 1
        while i != 0:
            w_lower = tuple(w[:i])
            Backoff = {1: L1, 2: L2, 3: L3, 4: L4}[i]
            if w_lower in Backoff.keys():
                p = Backoff[w_lower]
                break
            i -= 1
        else:
            # fallback uniform probability
            p = 1 / (sum(L1.values()) + len(L1))

        Katz = Alpha * p

    return Katz


In [38]:
def Greedy_Generate(Gram=1, L1=None, L2=None, L3=None, L4=None, G1=None, G2=None, G3=None, G4=None, l=100, end=20):
    L = {1: L1, 2: L2, 3: L3, 4: L4}[Gram]

    for i in range(l):
        Sentence = []

        starter = random.choice(list(L.keys()))
        Sentence.extend(starter)
        length = Gram

        while length < end:
            History = tuple(Sentence[-(Gram-1):]) if Gram > 1 else ()
            Max = float('-inf')
            Holder = None

            for x in L1.keys():
                Sen = History + (x,)
                if Sen in L:
                    if Max < L[Sen]:
                        Max = L[Sen]
                        Holder = x

            if Holder is None:
                break

            Sentence.append(Holder)
            length += 1

        print(" ".join(Sentence))


In [39]:
def Beam_Generate(Gram=1, L1=None, L2=None, L3=None, L4=None, G1=None, G2=None, G3=None, G4=None, l=100, end=20, beam_size=20):
    L = {1: L1, 2: L2, 3: L3, 4: L4}[Gram]

    for i in range(l):
        # Initialize beam with a random starter sequence
        starter = random.choice(list(L.keys()))
        beam = [(list(starter), 1.0)]  # list of tuples: (sequence, probability)

        length = len(starter)
        while length < end:
            new_beam = []
            for seq, prob in beam:
                History = tuple(seq[-(Gram-1):]) if Gram > 1 else ()
                for x in L1.keys():
                    Sen = History + (x,)
                    if Sen in L:
                        new_prob = prob * L[Sen]
                        new_beam.append((seq + [x], new_prob))

            if not new_beam:
                break

            # Keep top beam_size sequences
            new_beam.sort(key=lambda x: x[1], reverse=True)
            beam = new_beam[:beam_size]

            length += 1

        # Print the highest probability sequence in the beam
        best_seq = beam[0][0]
        print(" ".join(best_seq))

In [40]:
Para

[['అమెరికా',
  'అధ్యక్షుడు',
  'డొనాల్డ్',
  'ట్రంప్',
  'కు',
  'రాష్ట్రపతి',
  'భవన్',
  'వద్ద',
  'ఘనస్వాగతం',
  'లభించింది'],
 ['ఆయనకు',
  'రాష్ట్రపతి',
  'రామ్',
  'నాథ్',
  'కోవింద్',
  'దంపతులు',
  'ప్రధాని',
  'మోదీ',
  'సాదరంగా',
  'ఆహ్వానం',
  'పలకడంతో',
  'పాటు',
  'సైనికులు',
  'గౌరవ',
  'వందనాన్ని',
  'అందించారు'],
 ['ఇటు',
  'తెలంగాణలో',
  'కరోనా',
  'వైరస్',
  'కారణంగా',
  'అన్ని',
  'దేవాలయాల్లో',
  'ముందస్తు',
  'చర్యలు',
  'చేపట్టారు'],
 ['భద్రాద్రి', 'రాముడికి', 'కరోనా', 'ఎఫెక్ట్', 'తగిలింది'],
 ['ఏప్రిల్',
  '2న',
  'భద్రాద్రిలో',
  'జరగనున్న',
  'శ్రీరామ',
  'నవమి',
  'వేడుకలను',
  'వెంటాడుతోంది',
  'కరోనా'],
 ['రాష్ట్రంలో',
  'కొనసాగుతున్న',
  'కరోనా',
  'అలర్ట్',
  'నేపథ్యంలో',
  'భక్తులు',
  'లేకుండానే',
  'శ్రీరామనవమి',
  'జరుపుతామని',
  'మంత్రి',
  'పువ్వాడ',
  'అజయ్',
  'పేర్కొన్నారు'],
 ['ప్రత్యేక',
  'మీడియా',
  'సమావేశం',
  'ఏర్పాటు',
  'చేసిన',
  'మంత్రి',
  'పువ్వాడ',
  'ఈ',
  'మేరకు',
  'స్పష్టం',
  'చేశారు'],
 ['శ్రీ',
  'రాములవారి',
  'కల్యాణం',
  'టి

# Q1) Implement the Katz Backoff model for the quadrigram model.

In [41]:
Conut_List, Probability_List = NGram(4, Para[:15], Smoothing = None)

In [42]:
Conut_List_Quadrigram

NameError: name 'Conut_List_Quadrigram' is not defined

In [43]:
Conut_List[3]

{('అమెరికా', 'అధ్యక్షుడు', 'డొనాల్డ్', 'ట్రంప్'): 1,
 ('అధ్యక్షుడు', 'డొనాల్డ్', 'ట్రంప్', 'కు'): 1,
 ('డొనాల్డ్', 'ట్రంప్', 'కు', 'రాష్ట్రపతి'): 1,
 ('ట్రంప్', 'కు', 'రాష్ట్రపతి', 'భవన్'): 1,
 ('కు', 'రాష్ట్రపతి', 'భవన్', 'వద్ద'): 1,
 ('రాష్ట్రపతి', 'భవన్', 'వద్ద', 'ఘనస్వాగతం'): 1,
 ('భవన్', 'వద్ద', 'ఘనస్వాగతం', 'లభించింది'): 1,
 ('ఆయనకు', 'రాష్ట్రపతి', 'రామ్', 'నాథ్'): 1,
 ('రాష్ట్రపతి', 'రామ్', 'నాథ్', 'కోవింద్'): 1,
 ('రామ్', 'నాథ్', 'కోవింద్', 'దంపతులు'): 1,
 ('నాథ్', 'కోవింద్', 'దంపతులు', 'ప్రధాని'): 1,
 ('కోవింద్', 'దంపతులు', 'ప్రధాని', 'మోదీ'): 1,
 ('దంపతులు', 'ప్రధాని', 'మోదీ', 'సాదరంగా'): 1,
 ('ప్రధాని', 'మోదీ', 'సాదరంగా', 'ఆహ్వానం'): 1,
 ('మోదీ', 'సాదరంగా', 'ఆహ్వానం', 'పలకడంతో'): 1,
 ('సాదరంగా', 'ఆహ్వానం', 'పలకడంతో', 'పాటు'): 1,
 ('ఆహ్వానం', 'పలకడంతో', 'పాటు', 'సైనికులు'): 1,
 ('పలకడంతో', 'పాటు', 'సైనికులు', 'గౌరవ'): 1,
 ('పాటు', 'సైనికులు', 'గౌరవ', 'వందనాన్ని'): 1,
 ('సైనికులు', 'గౌరవ', 'వందనాన్ని', 'అందించారు'): 1,
 ('ఇటు', 'తెలంగాణలో', 'కరోనా', 'వైరస్'): 1,
 ('తెలంగాణలో',

In [51]:
Katz_dict = {}  # avoid naming conflict with the class name
for key in Conut_List[3].keys():
    Katz_dict[key] = Katz(
        Gram=4,
        L1=Conut_List[0],
        L2=Conut_List[1],
        L3=Conut_List[2],
        L4=Conut_List[3],
        G1=Probability_List[0],
        G2=Probability_List[1],
        G3=Probability_List[2],
        G4=Probability_List[3],
        Data=key
    )

(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89, 0, 1)
(1, 89

In [52]:
Katz_dict

{('అమెరికా', 'అధ్యక్షుడు', 'డొనాల్డ్', 'ట్రంప్'): 1.0,
 ('అధ్యక్షుడు', 'డొనాల్డ్', 'ట్రంప్', 'కు'): 1.0,
 ('డొనాల్డ్', 'ట్రంప్', 'కు', 'రాష్ట్రపతి'): 1.0,
 ('ట్రంప్', 'కు', 'రాష్ట్రపతి', 'భవన్'): 1.0,
 ('కు', 'రాష్ట్రపతి', 'భవన్', 'వద్ద'): 1.0,
 ('రాష్ట్రపతి', 'భవన్', 'వద్ద', 'ఘనస్వాగతం'): 1.0,
 ('భవన్', 'వద్ద', 'ఘనస్వాగతం', 'లభించింది'): 1.0,
 ('ఆయనకు', 'రాష్ట్రపతి', 'రామ్', 'నాథ్'): 1.0,
 ('రాష్ట్రపతి', 'రామ్', 'నాథ్', 'కోవింద్'): 1.0,
 ('రామ్', 'నాథ్', 'కోవింద్', 'దంపతులు'): 1.0,
 ('నాథ్', 'కోవింద్', 'దంపతులు', 'ప్రధాని'): 1.0,
 ('కోవింద్', 'దంపతులు', 'ప్రధాని', 'మోదీ'): 1.0,
 ('దంపతులు', 'ప్రధాని', 'మోదీ', 'సాదరంగా'): 1.0,
 ('ప్రధాని', 'మోదీ', 'సాదరంగా', 'ఆహ్వానం'): 1.0,
 ('మోదీ', 'సాదరంగా', 'ఆహ్వానం', 'పలకడంతో'): 1.0,
 ('సాదరంగా', 'ఆహ్వానం', 'పలకడంతో', 'పాటు'): 1.0,
 ('ఆహ్వానం', 'పలకడంతో', 'పాటు', 'సైనికులు'): 1.0,
 ('పలకడంతో', 'పాటు', 'సైనికులు', 'గౌరవ'): 1.0,
 ('పాటు', 'సైనికులు', 'గౌరవ', 'వందనాన్ని'): 1.0,
 ('సైనికులు', 'గౌరవ', 'వందనాన్ని', 'అందించారు'): 1.0,
 ('ఇటు', 'తెలంగాణల

# Q2. Implement the Kneser-Ney smoothing algorithm for the quadrigram model.

In [53]:
Kneser_dict = {}  # avoid naming conflict with the class name

for key in Conut_List[3].keys():
    Kneser_dict[key] = Kneser(
        Gram=4,
        Delta=0.7,
        L1=Conut_List[0],
        L2=Conut_List[1],
        L3=Conut_List[2],
        L4=Conut_List[3],
        G1=Probability_List[0],
        G2=Probability_List[1],
        G3=Probability_List[2],
        G4=Probability_List[3],
        Data=key
    )

In [54]:
Kneser_dict

{('అమెరికా', 'అధ్యక్షుడు', 'డొనాల్డ్', 'ట్రంప్'): 0.31123595505617985,
 ('అధ్యక్షుడు', 'డొనాల్డ్', 'ట్రంప్', 'కు'): 0.31123595505617985,
 ('డొనాల్డ్', 'ట్రంప్', 'కు', 'రాష్ట్రపతి'): 0.31123595505617985,
 ('ట్రంప్', 'కు', 'రాష్ట్రపతి', 'భవన్'): 0.31123595505617985,
 ('కు', 'రాష్ట్రపతి', 'భవన్', 'వద్ద'): 0.31123595505617985,
 ('రాష్ట్రపతి', 'భవన్', 'వద్ద', 'ఘనస్వాగతం'): 0.31123595505617985,
 ('భవన్', 'వద్ద', 'ఘనస్వాగతం', 'లభించింది'): 0.31123595505617985,
 ('ఆయనకు', 'రాష్ట్రపతి', 'రామ్', 'నాథ్'): 0.31123595505617985,
 ('రాష్ట్రపతి', 'రామ్', 'నాథ్', 'కోవింద్'): 0.31123595505617985,
 ('రామ్', 'నాథ్', 'కోవింద్', 'దంపతులు'): 0.31123595505617985,
 ('నాథ్', 'కోవింద్', 'దంపతులు', 'ప్రధాని'): 0.31123595505617985,
 ('కోవింద్', 'దంపతులు', 'ప్రధాని', 'మోదీ'): 0.31123595505617985,
 ('దంపతులు', 'ప్రధాని', 'మోదీ', 'సాదరంగా'): 0.31123595505617985,
 ('ప్రధాని', 'మోదీ', 'సాదరంగా', 'ఆహ్వానం'): 0.31123595505617985,
 ('మోదీ', 'సాదరంగా', 'ఆహ్వానం', 'పలకడంతో'): 0.31123595505617985,
 ('సాదరంగా', 'ఆహ్వానం', 'పల

# Q3. For each of the n-gram models generate 100 sentences using
- a. Greedy Approach (using maximum likelihood estimation)
- b. Beam Search with beam size=20

In [55]:
Greedy_Generate(
        Gram=4,
        L1=Conut_List[0],
        L2=Conut_List[1],
        L3=Conut_List[2],
        L4=Conut_List[3],
        G1=Probability_List[0],
        G2=Probability_List[1],
        G3=Probability_List[2],
        G4=Kneser_dict[key]
    )

ప్రధాని మోదీ సాదరంగా ఆహ్వానం పలకడంతో పాటు సైనికులు గౌరవ వందనాన్ని అందించారు
కరోనా అలర్ట్ నేపథ్యంలో భక్తులు లేకుండానే శ్రీరామనవమి జరుపుతామని మంత్రి పువ్వాడ అజయ్ పేర్కొన్నారు
ఆహ్వానం పలకడంతో పాటు సైనికులు గౌరవ వందనాన్ని అందించారు
సాదరంగా ఆహ్వానం పలకడంతో పాటు సైనికులు గౌరవ వందనాన్ని అందించారు
దేశాలను అభివృద్ది చెందిన దేశాలుగా భారత్ చైనాలతో పాటు మరికొన్ని దేశాలను అభివృద్ది చెందుతున్న దేశాలుగా ఆఫ్రికా దేశాలను వెనుకబడిన దేశాలకు పేర్కొంటున్నారు
అభివృద్ది చెందిన దేశాలుగా భారత్ చైనాలతో పాటు మరికొన్ని దేశాలను అభివృద్ది చెందుతున్న దేశాలుగా ఆఫ్రికా దేశాలను వెనుకబడిన దేశాలకు పేర్కొంటున్నారు
టికెట్లు రద్దు చేస్తున్నామని ప్రకటించారు
పలకడంతో పాటు సైనికులు గౌరవ వందనాన్ని అందించారు
శ్రీరామ నవమి వేడుకలను వెంటాడుతోంది కరోనా
ప్రధాని మోదీ సాదరంగా ఆహ్వానం పలకడంతో పాటు సైనికులు గౌరవ వందనాన్ని అందించారు
రాముడికి కరోనా ఎఫెక్ట్ తగిలింది
చాలా మంది విద్యుత్ పొదుపు చేయాలి అని విన్నాను
అన్ని దేవాలయాల్లో ముందస్తు చర్యలు చేపట్టారు
చేసిన మంత్రి పువ్వాడ ఈ మేరకు స్పష్టం చేశారు
దేశాలను వెనుకబడిన దేశాలకు పేర్కొంటున్నారు


In [56]:
Beam_Generate(
        Gram=4,
        L1=Conut_List[0],
        L2=Conut_List[1],
        L3=Conut_List[2],
        L4=Conut_List[3],
        G1=Probability_List[0],
        G2=Probability_List[1],
        G3=Probability_List[2],
        G4=Kneser_dict[key]
    )

చేసిన మంత్రి పువ్వాడ ఈ మేరకు స్పష్టం చేశారు
రాష్ట్రపతి రామ్ నాథ్ కోవింద్ దంపతులు ప్రధాని మోదీ సాదరంగా ఆహ్వానం పలకడంతో పాటు సైనికులు గౌరవ వందనాన్ని అందించారు
కొన్ని దేశాలను అభివృద్ది చెందిన దేశాలుగా భారత్ చైనాలతో పాటు మరికొన్ని దేశాలను అభివృద్ది చెందుతున్న దేశాలుగా ఆఫ్రికా దేశాలను వెనుకబడిన దేశాలకు పేర్కొంటున్నారు
నవమి వేడుకలను వెంటాడుతోంది కరోనా
అమెరికా సహా చాలా కొన్ని దేశాలను అభివృద్ది చెందిన దేశాలుగా భారత్ చైనాలతో పాటు మరికొన్ని దేశాలను అభివృద్ది చెందుతున్న దేశాలుగా ఆఫ్రికా దేశాలను వెనుకబడిన దేశాలకు
అభివృద్ది చెందిన దేశాలుగా భారత్ చైనాలతో పాటు మరికొన్ని దేశాలను అభివృద్ది చెందుతున్న దేశాలుగా ఆఫ్రికా దేశాలను వెనుకబడిన దేశాలకు పేర్కొంటున్నారు
ప్రధాని మోదీ సాదరంగా ఆహ్వానం పలకడంతో పాటు సైనికులు గౌరవ వందనాన్ని అందించారు
దేశాలను అభివృద్ది చెందిన దేశాలుగా భారత్ చైనాలతో పాటు మరికొన్ని దేశాలను అభివృద్ది చెందుతున్న దేశాలుగా ఆఫ్రికా దేశాలను వెనుకబడిన దేశాలకు పేర్కొంటున్నారు
భారత్ చైనాలతో పాటు మరికొన్ని దేశాలను అభివృద్ది చెందుతున్న దేశాలుగా ఆఫ్రికా దేశాలను వెనుకబడిన దేశాలకు పేర్కొంటున్నారు
చ

# 

# 